In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
import torchvision.models as models
from tqdm import tqdm

In [15]:
class HWIMGS(Dataset):
  def __init__(self, root, transform=None):
    " initial the dataset "
    self.image = None
    self.label = None
    self.filenames = []
    self.root = root
    self.transform = transform

    #read filenames
    filenames = glob.glob(root+'/*.png')
    for fn in filenames:
        #store data, label
        reg = fn
        
#         colab:/*.png
#         jupyter:\*.png
        reg = fn.split('\\')[-1].split('_')[0]
#         print(reg)
        reg = int(reg)
        self.filenames.append((fn, reg))

    self.len = len(filenames)

  def __getitem__(self, index):
    " get a sample from the dataset "
    img_fn, label = self.filenames[index]
    image = Image.open(img_fn)
    if self.transform is not None:
      image = self.transform(image)
    
    # if torch.cuda.is_available():
    #   img_fn, label = img_fn.cuda(), label.cuda()
    return image, label
  
  def __len__(self):
    " Total number of sampler in the dataset "
    return self.len

In [16]:
from torchvision import transforms
trainset = HWIMGS(root='p1_data/train_50', transform=transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation((-90,90)),
    # transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

valset = HWIMGS(root='p1_data/val_50', transform=transforms.Compose([
    transforms.Resize(224),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
print(len(trainset))
print(len(valset))

22500
2500


In [17]:
trainset_loader = DataLoader(trainset, batch_size=32, shuffle=True)
valset_loader = DataLoader(valset, batch_size=32, shuffle=False)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
from torchsummary import summary
resnet50 = models.resnet50(pretrained=True)



for id, c in enumerate(resnet50.children()):
    if id<7:
        for param in c.parameters():
            param.requires_grad = False
            

for id, c in enumerate(resnet50.children()):
    for param in c.parameters():
        print(param.requires_grad)
  

resnet50.fc= nn.Sequential(
    nn.Linear(2048, 50),
    # nn.ReLU(),
    # nn.Dropout(0.3),
    # nn.Linear(256, 128),
    # nn.ReLU(),
    # nn.Dropout(0.3),
    # nn.Linear(128, 50)
)


resnet50 = resnet50.to(device)
summary(resnet50, (3, 224, 224))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
----------------------------------------------------------------
 

In [20]:
for id, c in enumerate(resnet50.children()):
    print(id,c)

0 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
2 ReLU(inplace=True)
3 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
4 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [21]:
import matplotlib.pyplot as plt

train_acc_his, train_loss_his = [], []
val_acc_his, val_loss_his = [], []

In [22]:
def train(model, epoch, log_interval=100):
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    # optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion = nn.CrossEntropyLoss()
    model.train()  # Important: set training mode
    correct = 0

    for ep in range(epoch):
        correct = 0
        train_loss = 0
        cnt = 0
        for batch_idx, (data, target) in enumerate(tqdm(trainset_loader), 1):
            
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            train_loss += criterion(output, target).item()
            cnt+=1

        train_loss /= cnt
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {}/{} ({:.0f}%)'.format(
            ep+1, batch_idx * len(data), len(trainset_loader.dataset),
            100. * batch_idx / len(trainset_loader),train_loss,
            correct, len(trainset_loader.dataset),
            100. * correct / len(trainset_loader.dataset)))

        train_acc_his.append(100. * correct / len(trainset_loader.dataset))
        train_loss_his.append(train_loss)
        
        val(model,ep) # Evaluate at the end of each epoch
        # scheduler.step()
    # summarize history for accuracy
    plt.plot(train_acc_his)
    plt.plot(val_acc_his)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left') 
    plt.show()
    
    plt.plot(train_loss_his)
    plt.plot(val_loss_his)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left') 
    plt.show()

In [23]:
def val(model,ep):
# def val(model):
    criterion = nn.CrossEntropyLoss()
    model.eval()  # Important: set evaluation mode
    val_loss = 0
    correct = 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, target in valset_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(valset_loader)
    print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(valset_loader.dataset),
        100. * correct / len(valset_loader.dataset)))

    val_acc_his.append(100. * correct / len(valset_loader.dataset))
    val_loss_his.append(val_loss)
    if(100. * correct / len(valset_loader.dataset)>70):
        torch.save(resnet50, ('./{}_{:.0f}.pth').format(ep,100. * correct / len(valset_loader.dataset)),_use_new_zipfile_serialization=False)
    

In [11]:
# print(resnet50)

In [24]:
train(resnet50, epoch=30)
# val(resnet50,1)

100%|██████████| 704/704 [02:50<00:00,  4.13it/s]


Train Epoch: 1 [2816/22500 (100%)]	Loss: 1.859388, Accuracy: 10491/22500 (47%)


  0%|          | 0/704 [00:00<?, ?it/s]


Val set: Average loss: 1.1078, Accuracy: 1638/2500 (66%)



100%|██████████| 704/704 [02:45<00:00,  4.25it/s]


...

Train Epoch: 10 [2816/22500 (100%)]	Loss: 0.097419, Accuracy: 21785/22500 (97%)


  0%|          | 0/704 [00:00<?, ?it/s]


Val set: Average loss: 1.4848, Accuracy: 1788/2500 (72%)



 26%|██▋       | 186/704 [00:44<02:02,  4.21it/s]


KeyboardInterrupt: 

In [13]:
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)